In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_boston

In [51]:
boston = load_boston()
df = pd.DataFrame(boston.data)

In [52]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [53]:
df.columns = boston.feature_names
df['MEDV'] = boston.target

In [54]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [55]:
df.shape

(506, 14)

In [56]:
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]


In [57]:
y

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: MEDV, Length: 506, dtype: float64

In [58]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [59]:
X_train.shape

(404, 13)

In [60]:
rt = DecisionTreeRegressor(criterion='mse',max_depth=5)

In [61]:
rt.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=5)

In [62]:
y_pred = rt.predict(X_test)

In [63]:
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

0.8833565347917997

In [64]:
np.mean(cross_val_score(rt,X,y,scoring='r2',cv=5))

0.10752012221814651

## Hyper-Parameter Tunning

In [65]:

param_grid = {
    'max_depth':[2,4,8,10,None],
    'criterion':['mse','mae'],
    'max_features':[0.25,0.5,1.0],
    'min_samples_split':[0.25,0.5,1.0]
}
 

In [66]:
from sklearn.model_selection import GridSearchCV
reg = GridSearchCV(DecisionTreeRegressor(),param_grid=param_grid)
     

In [67]:
reg.fit(X_train,y_train)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': [2, 4, 8, 10, None],
                         'max_features': [0.25, 0.5, 1.0],
                         'min_samples_split': [0.25, 0.5, 1.0]})

In [68]:
reg.best_score_

0.6780727792544434

In [69]:
reg.best_params_

{'criterion': 'mse',
 'max_depth': 8,
 'max_features': 0.5,
 'min_samples_split': 0.25}

## Feature Importance

In [70]:
for importance, name in sorted(zip(rt.feature_importances_, X_train.columns),reverse=True):
    print(name,importance)

RM 0.6459193364379269
LSTAT 0.19426427075925418
DIS 0.07187288650018421
CRIM 0.0686217518012211
AGE 0.0061761261743666146
PTRATIO 0.005565056858702886
NOX 0.004410373376747717
INDUS 0.002627468726683442
B 0.0005427293649130764
ZN 0.0
TAX 0.0
RAD 0.0
CHAS 0.0
